<a href="https://colab.research.google.com/github/Amirhatamian/Statistical-Models-For-Data-Science/blob/main/Lesson3_27_11_2023_ToDo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Write your own Google drive path to files
DrivePath = "/content/drive/My Drive/Colab Notebooks"

# Link to Google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import math
import statistics
from scipy import stats
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# Specific importing
from pandas.plotting import lag_plot
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf

import warnings
warnings.filterwarnings("ignore")

#**1. Time Series Data Visualization in Python - Part 2**

##**1.1. Heatmaps**

A heatmap is a graphical representation of data where each value of a matrix is represented as a color. It can be useful to easily visualise how the distribution of a value evolves over time, as well as to visualise correlation (and covariance) values in a given dataset.

In [ ]:
# Example 1 - Temperature data
temperature_data = pd.read_csv(DrivePath +'/Data/Temperature.csv', na_values='',sep=';',header=None,names=['Date', 'Temp'], parse_dates=['Date'], index_col = 'Date')
display(temperature_data)

In [ ]:
!pip install july   # For defining heatmaps with days and months
import july # Similar package calplot
july.heatmap(temperature_data.index, temperature_data['Temp'].values, cmap='jet', colorbar=True, title='Average temperatures: Boston',month_grid=True);
# Note: if we add "value_label=True" we will visualise the value itself in each cell


In [ ]:
# Example 2 - Data on house sale prices for King County (America), for homes sold between May 2014-May 2015
houses = pd.read_csv(DrivePath +'/Data/kc_house_data.csv', na_values='',sep=',', parse_dates=['date'])
display(houses)

In [ ]:
# Heatmap to visualise the total number of houses sold per year/condition
houses_year_cond = pd.pivot_table(houses, values = 'id',index='condition', columns='yr_built',aggfunc='count')
display(houses_year_cond)
sns.heatmap(houses_year_cond, cmap='hot');

In [ ]:
# Note - Another useful plot for relating variables, besides scatterplot, is given by relplot

#fig, ax = plt.subplots(figsize=(10, 4));
#scatter = plt.scatter(houses['bedrooms'], houses['price'], c=houses['bathrooms'], cmap='viridis')
#legend1 = ax.legend(*scatter.legend_elements(), loc = 'center', bbox_to_anchor=(0.8, 0.5), title='Bathroom')
#ax.add_artist(legend1);
#ax.set(xlabel='Bedrooms', ylabel='Price',title='Scatter');

sns.relplot(x='bedrooms',y='price',hue='bathrooms',height=4,data=houses);

##**1.2 Boxplot and Violin plots**

Boxplots show the distribution of numeric data values. The box shows the quartiles of the dataset (Q1,Q3) while the whiskers extend to show the rest of the distribution (min, max), except for points that are determined to be outliers using the IQR (interquartile range) approach. \\
Violin plots are similar to boxplots, except that they also show the probability density of the data at different values. As in the standard boxplots, they include a marker for the median of the data and a box indicating the interquartile range. Overlaid is a kernel density estimation. They are both used to compare a variable distribution (or sample distribution) across different "categories".

In [ ]:
# Example 1 - Temperature data
temp = pd.read_csv(DrivePath +'/Data/daily-min-temperatures.csv', na_values='', sep = ';', parse_dates= ['Date'], index_col='Date', dayfirst=True)
temp['Period'] = temp.index.to_period('Y') # Add one colum to indicate the year (DatetimeIndex -> PeriodIndex)
display(temp)


In [ ]:
# Simple time plot for visualising the min temperature information
plt.figure(figsize=(10,3))
plt.grid()
plt.plot(temp['Temp'],'k-')
plt.title('Temperature')
plt.xlabel('Time [days]')
plt.ylabel('Values');

In [ ]:
# Boxplot representing the temperature data for each year
sns.boxplot(x=temp['Period'], y = temp['Temp'],palette='rainbow');

In [ ]:
# Boxplot representing the monthly temperature data for a specific year
temp_year = temp.loc[:'1981-12-31',:]
temp_year['Period'] = temp_year.index.to_period('M')
display(temp_year)

# Note: For Series type, we can use dt.strftime() function to do the conversion using specified date_format
# For example, '%b' - Month name, %m - Month number
sns.boxplot(x=temp_year['Period'].dt.strftime('%b'),y = temp_year['Temp'], palette='rainbow');


In [ ]:
# Violin plot
sns.violinplot(x=temp_year['Period'].dt.strftime('%b'),y = temp_year['Temp'], palette='rainbow');

In [ ]:
# Note: IQR for defining an outlier

P = [10,300,450,470,550,350,320,1000]
Q1_py = np.quantile(P,0.25)
Q3_py = np.quantile(P,0.75)
IQR = Q3_py-Q1_py
Lower_Fence = Q1_py - 1.5*IQR
Upper_Fence = Q3_py + 1.5*IQR

T = []
for i in P:
    exp1 = i < Lower_Fence
    exp2 = i > Upper_Fence
    if exp1 or exp2:
       temp = i
       T.append(temp)

print('Outliers:', T)

plt.figure(figsize=(3,4))
ax = sns.boxplot(y = P, palette='rainbow');
ax.grid()

When the distribution of values in the sample is Gaussian or Gaussian-like, the standard deviation of the sample can be used as a cut-off for identifying outliers. In particular, 3*standard deviations from the mean will account for the 99.7% of the sample data (2 * standard deviations 95%).

In [ ]:
# Standard deviation approach for defining outliers
np.random.seed(42)
P = 5 * np.random.randn(10000) + 50

mean_value = np.mean(P)
sd_value = np.std(P)
thr = 3*sd_value
Lower = mean_value - thr
Upper = mean_value + thr

outliers = [i for i in P if i < Lower or i > Upper]
print('Number of outliers:', len(outliers))

P_no_outliers = [i for i in P if i > Lower and i < Upper]
print('Number of non-outliers:', len(P_no_outliers))

plt.figure(figsize=(3,4))
ax = sns.boxplot(y = P, palette='rainbow');
ax.grid()

##**1.3 Barplots and Histograms**
A barplot visualizes data with rectangular bars with height proportional to the values that they represent. There are different types of barplots, according to the information they convey:
> - Simple Bar Plot: an item for each category is shown by plotting bars of equal width but variable length;

> - Grouped Bar Plot: the bars for categorical variable values are very close to each other, and hence the name. This type of plot is used for plotting a set of entities split in groups and subgroups;

> - Stacked Bar Plot: bars represent the sub-groups, and are placed on top of each other to form a single column (or say, single bar). The overall length of the bar gives the total size of the category, and different colors indicate their relative contribution to each sub-group.


In [ ]:
# Example 1 - Temperature data (Yearly and Monthly)
# Barplot
temp = pd.read_csv(DrivePath +'/Data/daily-min-temperatures.csv', na_values='', sep = ';', parse_dates= ['Date'], index_col='Date', dayfirst=True)
temp['Period'] = temp.index.to_period('Y') # Add one colum to indicate the year (DatetimeIndex -> PeriodIndex)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.barplot(ax = axes[0], x=temp['Period'].dt.strftime('%Y'), y= temp['Temp'],palette='autumn',errorbar=('se'),capsize=0.01,estimator='mean')
axes[0].set_title('Yearly Data', fontsize=12)
axes[0].grid()

sns.barplot(ax = axes[1], x=temp_year['Period'].dt.strftime('%b'), y= temp_year['Temp'],palette='autumn',errorbar=('se'),capsize=0.01,estimator='mean')
axes[1].set_title('Monthly Data - 1981', fontsize=12)
axes[1].grid();


Simple histograms can represent a good first step for understanding a dataset, allowing to show the frequency of numerical data. The `hist()` function requires only a single argument, that is an array of elements. It can return:
> - *n*: heights of the histogram bins \\
> - *bins*: edges of the bins’ base \\
> - *patches*: containers of individual artists used to create the histogram \\

Among the optional arguments important are bins (i.e., how many bins to use to divide data) and density (if True, returns a probability density. Each bin will thus display the bins count divided by the total number of counts and the bin width. In this way, area under the histogram integrates to 1).

In [ ]:
# Histogram - Counts/Frequency (Daily data for 1981)
n, bins, patches = plt.hist(temp_year['Temp'], bins = 20, density=False) # n = the number of points in a given bin
plt.grid()
print('The total counts is', sum(n)) # sum of the bins heights

In [ ]:
# Histogram -  Density
n, bins, patches = plt.hist(temp_year['Temp'], bins = 20, density=True) # n = count_bin/(width_bin*total_counts)
plt.grid()
a = np.diff(bins) # to get the width of each bin
print('The total area is', sum(n*a[0])) # sum of the bins' areas

A histogram aims to approximate the underlying probability density function that generated the data by binning and counting observations. Kernel density estimation (KDE) presents a different solution, where rather than using discrete bins, it smooths the observations with a Gaussian kernel, producing a continuous density estimate:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 3))
sns.histplot(temp_year['Temp'],kde=True,color='red',alpha=0.2,stat='probability',binwidth=1,fill=True, ax=axes[0]);
axes[0].grid()
axes[0].set_xlim([0, 25]);
axes[0].set_ylim([0, 0.09]);

sns.kdeplot(temp_year['Temp'],color='r', shade=True, ax=axes[1])
axes[1].grid()
axes[1].set_xlim([0, 25]);
axes[1].set_ylim([0, 0.09]);

##**Exercise 1 - Temperature & CO2**

Consider the two csv files named "CO2.csv" and "GLB.Ts+dSST.csv". The *CO2.csv* file contains the corresponding data from the US National Oceanic and Atmospheric Administration, and *Trend* is the variable we are interested in (note: be careful with the date information). *GLB.Ts+dSST.csv* instead includes the global temperature anomalies (combined land-surface air and sea-surface water temperature anomalies - land-ocean temperature index, L-OTI) expressed as deviations from the corresponding 1951-1980 means (https://data.giss.nasa.gov/gistemp/). \\
After having loaded the data and explored the datasets, convert the data in order to have yearly frequency (average as aggregator). Perform all the necessary operations required to calculate and then visualise in an appropriate graph the correlation between the yearly time courses of CO2 and global temperature anomalies.

##**Exercise 2 - Brain Activity and Correlation**

Consider the *Ab_pASL_Yeo_Average.txt* file which contains the brain time courses for 100 different regions and 200 time points.
After having loaded the data, perform the following operations: \\
1. Visualise in the same figure the signals for regions 3 and 4;
2. Subtract the temporal mean from signal 3 (and the same for signal 4), and plot together in a new figure the demeaned signals;
3. Calculate the correlation coefficient for each pair of time courses, and then the covariance. What is the resulting dimension of these two operations?
4. Visualize the two results from step 3 using the most appropriate graph. In addition, in another graph, visualize the distribution of the correlation values just calculated;
5. Calculate the correlation between the correlation coefficient values and the covariance ones.  

##**1.4 Lag plots**

A lag plot checks whether a time series is random or not. Random data should not exhibit any identifiable structure in the lag plot, while non-random structure in the lag plot indicates that the underlying data are correlated (not random).
A lag is a fixed time displacement. A plot of lag 1 is a plot of the values of Yi versus Yi+1, thus a lag plot is essentially a scatter plot with the two time series properly lagged.

In [ ]:
# Example with yearly data on Australian beer production
df = pd.read_csv(DrivePath +'/Data/Australian_Beer_production.csv', na_values='',sep=';', parse_dates=['time'],index_col='time')
signal = df['value']
plt.plot(signal,marker='.')
plt.grid()
plt.xlabel('Time')
plt.ylabel('Values');


In [ ]:
# Lagged version of the original signal
mysignals = [{'name': 'Lag0', 'y': signal,'color':'g', 'linewidth':2},
              {'name': 'Lag1', 'y': signal.shift(1),'color':'r', 'linewidth':2},
            {'name': 'Lag2', 'y':  signal.shift(2),'color':'b', 'linewidth':2}]

fig, ax = plt.subplots(figsize=(6,5))
for line_lag in mysignals:
    ax.plot(line_lag['y'],
            color=line_lag['color'],
            linewidth=line_lag['linewidth'],
            label=line_lag['name'])

ax.grid()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title('Representative Lags');

In [ ]:
# Lag plots
fig, axes = plt.subplots(3,3, sharex=True, sharey=True, figsize=(8,8))

for i, ax in enumerate(axes.flatten()[:9]):
    pd.plotting.lag_plot(signal, lag=i+1, ax=ax, c='black')
    ax.grid()
    ax.set_xlabel('y(t)')
    ax.set_ylabel('y(t+'+str(i+1)+')')
    pt = (350, 350)
    ax.axline(pt, slope=1, color='gray')



##**1.4.1 Autocorrelation**

Just as correlation correlation measures the extent of a linear relationship between two variables, autocorrelation measures the linear relationship between lagged values of a time series. There are several autocorrelation coefficients, corresponding to each panel in the lag plot. For example, r1 measures the relationship between yt and yt+1, r2 measures the relationship between y and yt+2, and so on.
The autocorrelation coefficients make up the *autocorrelation function* or *ACF*.

In [ ]:
# Example 1 - Simulated Signal
a = pd.Series([10, 15, 30, 40, 60])
b = a.shift(1)[1:]
display(a)
display(b)


In [ ]:
# General simple equation to calculate the autocorrelation
a2 = a[1:]-np.mean(a)
b2 = b-np.mean(a)
den = a-np.mean(a)
res = sum(a2*b2)/sum(den**2)
display(res)

# Double check with automatic Python function:
val = acf(a,nlags=4)
display(val)

In [ ]:
# Automatic plot of the ACF function for the previous signal (Australian beer production)
fig, ax = plt.subplots(figsize=(8,3))
plot_acf(signal, lags=9,ax=ax)
ax.grid()

# Note: to retrieve the raw values resulting from this operation, we can use acf():
val = acf(signal,nlags=9)
display(val)

# val1 indicates how successive values of the signals relate to each other
# val2 indicates how signal values 2 period aparts relate to each other

This plot is often known as correlogram. The shaded indicate whether the correlations are significantly different from zero and so if there is evidence of no autocorrelation structure.

##**1.4.2 White Noise**

Time series that show no autocorrelation are called **white noise**. White noise data is uncorrelated across time with zero mean and constant variance.As such, white noise variations in the data cannot be explained by any model. We should regard this as something out of interest and not predictable. \\
When looking at the autocorrelation results for different lags, all the values should be close to zero.

In [ ]:
# Random signal and Lag plot
np.random.seed(42)

mean = 0
std = 1
num_samples = 1000
samples = pd.Series(np.random.normal(mean, std, size=num_samples))

fig, axes = plt.subplots(figsize=(3,3))
plt.plot(samples)
plt.grid()
plt.title('Random signal')

fig, axes = plt.subplots(3,3, sharex=True, sharey=True, figsize=(8,8))

for i, ax in enumerate(axes.flatten()[:9]):
    pd.plotting.lag_plot(samples, lag=i+1, ax=ax, c='black')
    ax.grid()
    ax.set_xlabel('y(t)')
    ax.set_ylabel('y(t+'+str(i+1)+')')
    pt = (-4, -4)
    ax.axline(pt, slope=1, color='gray')

In [ ]:
# Autocorrelation of the random signal
fig, ax = plt.subplots(figsize=(8,4))
plot_acf(samples, lags=9,ax=ax)
ax.grid()

val = acf(samples,nlags=9)
display(val)

##**(Short) Exercise 3 - Google Stock Price**
After having loaded the Google Stock Prices from beginning 2004 till the end of 2016 with monthly frequency, plot the data and the autocorrelation plot with the corresponding  values (note: consider as maximum value for the autocorrelation lags within one year). How does it look like? \\
Calculate then the differencing with the previous month. What happens to signal and to the the autocorrelation values?



##**Exercise Extra  - Energy**

In this exercise, load the data contained in the "Energy_consumption.csv" file. This contains information related to the Germany country-wide totals of electricity consumption, wind power production, and solar power production (wind+solar is also reported).

In [ ]:
# Load the data, dropping all the rows with NaN values and the Wind+Solar column


In [ ]:
# Resample the dates with a frequency of 15 days, taking mean as aggregator, and visualise the time course of the total Consumption over years


In [ ]:
# Prepare a figure with three subplots. In each subplot, illustrate the distribution of the total Consumption, Wind and Solar production values, respectively,
# grouped by year


In [ ]:
# Represent with an appropriate graph the correlation between Wind and Consumption time courses


In [ ]:
# Calculate two matrices (dimension=3x3), one representing the Pearson correlation coefficient values for each pair of signals
# and the other one the corresponding p-values. Visualise both of them with appropriate graphs
